In [ ]:
!pip install tweepy
!pip install textblob

In [2]:
import re # regular expression / regex
import tweepy #Collect tweets
from tweepy import OAuthHandler # Authentication
from textblob import TextBlob # provides text mining, text analysis and text processing modules

In [3]:
class TwitterClient(object):
    '''
    Generic Twitter Class for sentiment analysis.
    '''
    def __init__(self):
        '''
        Class constructor or initialization method.
        '''
        # keys and tokens from the Twitter Dev Console
        consumer_key = 'ts8RhO1B1qIzdSLaDsaise89a'
        consumer_secret = 'A4ZawHMrKiAaoxfQhmjrW5F2gWwgjh9jhCuvczy8cymLaTJfES'
        access_token = '929606650415136768-Ok0ca8lkQk7oD8xNUqKWKg9VyoEXJRV'
        access_token_secret = '5mJm07yiPpvFB7M4Z9l4yaip4k6gvSpL2mEMMs6sEXjqX'
 
        # attempt authentication
        try:
            # create OAuthHandler object
            self.auth = OAuthHandler(consumer_key, consumer_secret)
            # set access token and secret
            self.auth.set_access_token(access_token, access_token_secret)
            # create tweepy API object to fetch tweets
            self.api = tweepy.API(self.auth)
        except:
            print("Error: Authentication Failed")
 
    def clean_tweet(self, tweet):
        '''
        Utility function to clean tweet text by removing links, special characters
        using simple regex statements.
        '''
        return ' '.join(re.sub("/@%&", " ", tweet).split())
 
    def get_tweet_sentiment(self, tweet):
        '''
        Utility function to classify sentiment of passed tweet
        using textblob's sentiment method
        '''
        # create TextBlob object of passed tweet text
        analysis = TextBlob(self.clean_tweet(tweet))
        # set sentiment
        if analysis.sentiment.polarity > 0:
            return 'positive'
        elif analysis.sentiment.polarity == 0:
            return 'neutral'
        else:
            return 'negative'
 
    def get_tweets(self, query, count = 1000):
        '''
        Main function to fetch tweets and parse them.
        '''
        # empty list to store parsed tweets
        tweets = []
 
        try:
            # call twitter api to fetch tweets
            fetched_tweets = self.api.search(q = query, count = count)
 
            # parsing tweets one by one
            for tweet in fetched_tweets:
                # empty dictionary to store required params of a tweet
                parsed_tweet = {}
 
                # saving text of tweet
                parsed_tweet['text'] = tweet.text
                # saving sentiment of tweet
                parsed_tweet['sentiment'] = self.get_tweet_sentiment(tweet.text)
 
                # appending parsed tweet to tweets list
                if tweet.retweet_count > 0:
                    # if tweet has retweets, ensure that it is appended only once
                    if parsed_tweet not in tweets:
                        tweets.append(parsed_tweet)
                else:
                    tweets.append(parsed_tweet)
 
            # return parsed tweets
            return tweets
 
        except tweepy.TweepError as e:
            # print error (if any)
            print("Error : " + str(e))

In [4]:
def main(querytext):
    # creating object of TwitterClient Class
    api = TwitterClient()
    # calling function to get tweets
    tweets = api.get_tweets(query = querytext, count = 1000)
    print("Queries Text : ", querytext)
    # picking positive tweets from tweets
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive']
    # percentage of positive tweets
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets)))
    # picking negative tweets from tweets
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative']
    # percentage of negative tweets
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets)))
    # percentage of neutral tweets
    print("Neutral tweets percentage: {} %".format(100*(len(tweets) - len(ntweets) - len(ptweets))/len(tweets)))
    print("\n\n Total Tweets:",len(tweets))
    # printing first 5 positive tweets
    print("\n\nPositive tweets:")
    for tweet in ptweets[:5]:
        print(tweet['text'])
 
    # printing first 5 negative tweets
    print("\n\nNegative tweets:")
    for tweet in ntweets[:5]:
        print(tweet['text'])

In [5]:
main('trump')

Queries Text :  trump
Positive tweets percentage: 28.571428571428573 %
Negative tweets percentage: 25.510204081632654 %
Neutral tweets percentage: 45.91836734693877 %


 Total Tweets: 98


Positive tweets:
RT @mrjamesob: ‘How dare you call me racist! I’m a huge fan of Donald Trump &amp; he’s bright orange.
@DoremusSchafer @KarlBode @archnerdian @mbaram Ever thought that some people might have thought it was funny to tag… https://t.co/e2EwlHV4aF
Donald Trump sees sharp increase in support for economic policies https://t.co/ljQUeRNfRF via @financialtimes
RT @ArmyBratt68: Good Morning Cult45 MAGA Team Trump!  Happy Monday Fun day!!
RT @joshtpm: Fascinating dimension of the Trump era: all news orgs now hitting live with Trumps claim about four embassies and considering…


Negative tweets:
RT @JamieGlazov: Lopez: Behind the Scenes of the Suleymani Hit.
The real reason Trump eradicated the terror master.
https://t.co/W7oNCpp5hX
RT @impeach_45NOW: “The difficulty that we have, and I don’t mea